In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
from fastai import * 
from fastai.vision import *

from pathlib import Path
import json 
from matplotlib import patches, patheffects
import os 
import collections
import pandas as pd

In [18]:
imgs = [] 
img_file = open('data/boxes.csv', 'r')
for line in img_file: 
    en = line.strip().split(',') 
    imgs.append( (en[0], [int(i) for i in en[1].split()] , en[2]))

In [19]:
imgs

[('imgs/img1.jpg', [121, 16, 492, 390], '0'),
 ('imgs/img2.jpg', [1057, 396, 2100, 1450], '1'),
 ('imgs/img3.jpg', [213, 39, 673, 519], '0')]

In [31]:
df = pd.DataFrame( {'fn': [ i[0] for i in imgs ] , 
                    'x0': [ i[1][0] for i in imgs],
                    'y0': [ i[1][1] for i in imgs],
                    'x1': [ i[1][2] for i in imgs],
                    'y1': [ i[1][3] for i in imgs],
                    'cat': [ i[2] for i in imgs ] 
                   }, columns = ['fn', 'x0', 'y0', 'x1', 'y1', 'cat'] )
df.to_csv('data/dataset.csv', index=False)

In [21]:
class ObjectList(ObjectCategoryList):
    "`ItemList` for labelled bounding boxes."
    _processor = vision.data.ObjectCategoryProcessor
    
    def get(self, i):
        # I imagine self.items[i] returns only one bounding box here, as the list of columns['x0', 'y0', 'x1', 'y1']
        return ImageBBox.create(*_get_size(self.x,i), [self.items[i]], labels=[self.classes[0]], classes=self.classes, pad_idx=self.pad_idx)

In [40]:
class ObjectList(ObjectCategoryList):
    "`ItemList` for labelled bounding boxes."
    _processor = vision.data.ObjectCategoryProcessor

    def get(self, i):
        # I imagine self.items[i] returns only one bounding box here, as the list of columns + label['x0', 'y0', 'x1', 'y1', 'label']
        bb = self.items[i]
        return ImageBBox.create(*_get_size(self.x,i), [bb[:-1]], labels=[bb[-1]], classes=self.classes, pad_idx=self.pad_idx)

data = ObjectItemList.from_csv(path='data', 
                         csv_name='dataset.csv', 
                         folder='imgs',
                         )
       # .split_none()
       # .label_from_df(cols=[ 'x0', 'y0', 'x1', 'y1', 'cat'], label_cls=ObjectList)
       # .transform(None, resize_method = ResizeMethod.SQUISH, size=224, tfm_y=True)
       # .databunch(bs=64) )

In [41]:
data

ObjectItemList (3 items)
Image (3, 400, 600),Image (3, 2149, 3224),Image (3, 528, 793)
Path: data